In [1]:
import matplotlib.pyplot as plt
import numpy as np
from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit.visualization import plot_histogram
from math import gcd
from numpy.random import randint
import pandas as pd
from fractions import Fraction
from math import gcd # greatest common divisor


In [12]:
def c_amod15(a, power):
    """Controlled multiplication by a mod 15"""
    if a not in [2,4,7,8,11,13]:
        raise ValueError("'a' must be 2,4,7,8,11 or 13")
    U = QuantumCircuit(4)        
    for iteration in range(power):
        if a in [2,13]:
            U.swap(0,1)
            U.swap(1,2)
            U.swap(2,3)
        if a in [7,8]:
            U.swap(2,3)
            U.swap(1,2)
            U.swap(0,1)
        if a in [4, 11]:
            U.swap(1,3)
            U.swap(0,2)
        if a in [7,11,13]:
            for q in range(4):
                U.x(q)
    U = U.to_gate()
    U.name = "%i^%i mod 15" % (a, power)
    c_U = U.control()
    return c_U


In [13]:
def qft_dagger(n):
    """n-qubit QFTdagger the first n qubits in circ"""
    qc = QuantumCircuit(n)
    # Don't forget the Swaps!
    for qubit in range(n//2):
        qc.swap(qubit, n-qubit-1)
    for j in range(n):
        for m in range(j):
            qc.cp(-np.pi/float(2**(j-m)), m, j)
        qc.h(j)
    qc.name = "QFT†"
    return qc

In [14]:
def a2jmodN(a, j, N):
    """Compute a^{2^j} (mod N) by repeated squaring"""
    for i in range(j):
        a = np.mod(a**2, N)
    return a


In [15]:
def qpe_amod15(a):
    n_count = 8
    qc = QuantumCircuit(4+n_count, n_count)
    for q in range(n_count):
        qc.h(q)     # Initialize counting qubits in state |+>
    qc.x(3+n_count) # And auxiliary register in state |1>
    for q in range(n_count): # Do controlled-U operations
        qc.append(c_amod15(a, 2**q), 
                 [q] + [i+n_count for i in range(4)])
    qc.append(qft_dagger(n_count), range(n_count)) # Do inverse-QFT
    qc.measure(range(n_count), range(n_count))
    (qc.draw(fold=-1))
    # Simulate Results
    aer_sim = Aer.get_backend('aer_simulator')
    # Setting memory=True below allows us to see a list of each sequential reading
    t_qc = transpile(qc, aer_sim)
    result = aer_sim.run(t_qc, shots=1, memory=True).result()
    readings = result.get_memory()
    print("Register Reading: " + readings[0])
    phase = int(readings[0],2)/(2**n_count)
    print("Corresponding Phase: %f" % phase)
    return phase

In [ ]:
n_count = 8  # number of counting qubits
a = 7

# Create QuantumCircuit with n_count counting qubits
# plus 4 qubits for U to act on
qc = QuantumCircuit(n_count + 4, n_count)

# Initialize counting qubits
# in state |+>
for q in range(n_count):
    qc.h(q)
    
# And auxiliary register in state |1>
qc.x(3+n_count)

# Do controlled-U operations
for q in range(n_count):
    qc.append(c_amod15(a, 2**q), 
             [q] + [i+n_count for i in range(4)])

# Do inverse-QFT
qc.append(qft_dagger(n_count), range(n_count))

# Measure circuit
qc.measure(range(n_count), range(n_count))
qc.draw(fold=-1)  # -1 means 'do not fold'

aer_sim = Aer.get_backend('aer_simulator')
t_qc = transpile(qc, aer_sim)
results = aer_sim.run(t_qc).result()
counts = results.get_counts()
plot_histogram(counts)

rows, measured_phases = [], []
for output in counts:
    decimal = int(output, 2)  # Convert (base 2) string to decimal
    phase = decimal/(2**n_count)  # Find corresponding eigenvalue
    measured_phases.append(phase)
    # Add these values to the rows in our table:
    rows.append([f"{output}(bin) = {decimal:>3}(dec)", 
                 f"{decimal}/{2**n_count} = {phase:.2f}"])
# Print the rows in a table
headers=["Register Output", "Phase"]
df = pd.DataFrame(rows, columns=headers)
print(df)

rows = []
for phase in measured_phases:
    frac = Fraction(phase).limit_denominator(15)
    rows.append([phase, f"{frac.numerator}/{frac.denominator}", frac.denominator])
# Print as a table
headers=["Phase", "Fraction", "Guess for r"]
df = pd.DataFrame(rows, columns=headers)
print(df)

a2jmodN(7, 2049, 53)

phase = qpe_amod15(a) # Phase = s/r
Fraction(phase).limit_denominator(15) 

frac = Fraction(phase).limit_denominator(15)
s, r = frac.numerator, frac.denominator
print(r)

print("is this N?")
print(N)

guesses = [gcd(a**(r//2)-1, N), gcd(a**(r//2)+1, N)]
print(guesses)

a = 7
factor_found = False
attempt = 0
while not factor_found:
    attempt += 1
    print("\nAttempt %i:" % attempt)
    phase = qpe_amod15(a) # Phase = s/r
    frac = Fraction(phase).limit_denominator(N) # Denominator should (hopefully!) tell us r
    r = frac.denominator
    print("Result: r = %i" % r)
    if phase != 0:
        # Guesses for factors are gcd(x^{r/2} ±1 , 15)
        guesses = [gcd(a**(r//2)-1, N), gcd(a**(r//2)+1, N)]
        print("Guessed Factors: %i and %i" % (guesses[0], guesses[1]))
        for guess in guesses:
            if guess not in [1,N] and (N % guess) == 0: # Check to see if guess is a factor
                print("*** Non-trivial factor found: %i ***" % guess)
                factor_found = True

            Register Output         Phase
0  00000000(bin) =   0(dec)  0/256 = 0.00
   Phase Fraction  Guess for r
0    0.0      0/1            1
Register Reading: 00000000
Corresponding Phase: 0.000000
1
is this N?
15
[15, 1]

Attempt 1:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 2:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 3:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 4:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 5:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 6:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 7:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 8:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 9:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 10:
Register Read

/Users/macbookair/Desktop/quantum_hackathon/virtual/environment/lib/python3.10/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/macbookair/Desktop/quantum_hackathon/virtual/environment/lib/python3.10/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 22:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 23:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 24:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 25:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 26:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 27:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 28:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 29:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 30:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 31:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 32:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Atte

Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 134:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 135:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 136:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 137:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 138:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 139:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 140:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 141:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 142:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 143:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 144:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: 

Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 248:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 249:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 250:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 251:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 252:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 253:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 254:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 255:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 256:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 257:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 258:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: 

...

Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 4765:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 4766:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 4767:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 4768:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 4769:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 4770:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 4771:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 4772:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 4773:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 4774:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 4775:
Register Reading: 00000000
Corresponding Phase: 0.0000

Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 4868:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 4869:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 4870:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 4871:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 4872:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 4873:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 4874:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 4875:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 4876:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 4877:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 4878:
Register Reading: 00000000
Corresponding Phase: 0.0000

Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 4975:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 4976:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 4977:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 4978:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 4979:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 4980:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 4981:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 4982:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 4983:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 4984:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 4985:
Register Reading: 00000000
Corresponding Phase: 0.0000

Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 5085:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 5086:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 5087:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 5088:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 5089:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 5090:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 5091:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 5092:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 5093:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 5094:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 5095:
Register Reading: 00000000
Corresponding Phase: 0.0000

Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 5192:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 5193:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 5194:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 5195:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 5196:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 5197:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 5198:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 5199:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 5200:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 5201:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 5202:
Register Reading: 00000000
Corresponding Phase: 0.0000

In [ ]:
# Measure circuit
qc.measure(range(n_count), range(n_count))
qc.draw(fold=-1)  # -1 means 'do not fold'

In [7]:
a = 7
N = 15

phase = qpe_amod15(a) # Phase = s/r

Fraction(phase).limit_denominator(15) 

frac = Fraction(phase).limit_denominator(15)
s, r = frac.numerator, frac.denominator
print(r)

print("is this N?")
print(N)

guesses = [gcd(a**(r//2)-1, N), gcd(a**(r//2)+1, N)]
print(guesses)

a = 7
factor_found = False
attempt = 0
while not factor_found:
    attempt += 1
    print("\nAttempt %i:" % attempt)
    phase = qpe_amod15(a) # Phase = s/r
    frac = Fraction(phase).limit_denominator(N) # Denominator should (hopefully!) tell us r
    r = frac.denominator
    print("Result: r = %i" % r)
    if phase != 0:
        # Guesses for factors are gcd(x^{r/2} ±1 , 15)
        guesses = [gcd(a**(r//2)-1, N), gcd(a**(r//2)+1, N)]
        print("Guessed Factors: %i and %i" % (guesses[0], guesses[1]))
        for guess in guesses:
            if guess not in [1,N] and (N % guess) == 0: # Check to see if guess is a factor
                print("*** Non-trivial factor found: %i ***" % guess)
                factor_found = True

Register Reading: 10000000
Corresponding Phase: 0.500000
2
is this N?
15
[3, 1]

Attempt 1:
Register Reading: 00000000
Corresponding Phase: 0.000000
Result: r = 1

Attempt 2:
Register Reading: 10000000
Corresponding Phase: 0.500000
Result: r = 2
Guessed Factors: 3 and 1
*** Non-trivial factor found: 3 ***


In [8]:
qpe_amod15(a)

Register Reading: 00000000
Corresponding Phase: 0.000000


0.0

In [9]:
i = 0
while i < 10:
    print((2**(i))%15)
    i = i + 1

1
2
4
8
1
2
4
8
1
2
